## Code For Fully Frozen Bert Weights (Regression)
- The patients here have not been separated based on whether they died

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from multimodal import MultimodalDataset, LOSNetWeighted, collation
from sklearn.preprocessing import StandardScaler
from transformers import AutoModel
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [2]:
!nvidia-smi

Sun Apr  7 09:19:01 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   47C    P8    23W / 300W |   1641MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
base_path = 'data/regression/with-outliers/combined'

static_train = pd.read_csv(f'{base_path}/static_train.csv')
static_val = pd.read_csv(f'{base_path}/static_val.csv')
static_test = pd.read_csv(f'{base_path}/static_test.csv')

In [4]:
static_train.shape

(7853, 24)

In [5]:
notes = pd.read_csv('data/notes_cleaned.csv')
notes_train = notes[notes['id'].isin(static_train['id'])]
notes_val = notes[notes['id'].isin(static_val['id'])]
notes_test = notes[notes['id'].isin(static_test['id'])]

In [6]:
dynamic = pd.read_csv('data/dynamic_cleaned.csv')
dynamic_train = dynamic[dynamic['id'].isin(static_train['id'])].copy()
dynamic_val = dynamic[dynamic['id'].isin(static_val['id'])].copy()
dynamic_test = dynamic[dynamic['id'].isin(static_test['id'])].copy()

In [7]:
features = ['aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium']

scaler = StandardScaler()

dynamic_train.loc[:, features] = scaler.fit_transform(dynamic_train[features])
dynamic_val.loc[:, features] = scaler.transform(dynamic_val[features])
dynamic_test.loc[:, features] = scaler.transform(dynamic_test[features])  

dynamic_train.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium
0,28793466,4/12/29 3:35,0.071185,-0.349005,-0.906293,0.416715,0.575845,-0.712386,0.059663,0.297403,-0.639959
1,26115624,9/7/50 0:22,-0.751069,-0.167053,-1.018477,-0.522606,1.260931,-0.712386,-0.606627,0.647561,-0.900545
3,20329785,5/12/34 19:32,-0.956633,2.744188,-0.195790,1.773512,-1.890467,-0.617347,-0.630005,-0.753070,-0.379374
4,24566943,6/25/55 15:45,0.071185,-0.167053,0.327738,0.312346,0.575845,0.047929,0.281760,0.472482,-0.379374
5,21792938,4/13/28 14:18,-0.956633,0.924662,0.365133,-1.357559,-0.794329,1.378482,-0.630005,-1.278306,1.184140


### Dynamic train preprocessing

In [8]:
id_lengths_train = dynamic_train['id'].value_counts().to_dict()
dynamic_train = dynamic_train.sort_values(by=['id', 'charttime'])
dynamic_train = dynamic_train.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_train['id']).agg(list)

dynamic_train

id
20001305    [[-0.545505639426621, 0.37880475678178827, 0.4...
20001361    [[-0.33994208065368914, -0.16705291664414468, ...
20003425    [[-1.5733234332912807, 1.1066149880163654, -0....
20008098    [[0.48231215443803854, -0.7129105900700776, -0...
20009330    [[0.2767485956651066, -1.0768157056873662, -0....
                                  ...                        
29991038    [[0.48231215443803854, -0.7129105900700776, 1....
29991539    [[0.48231215443803854, 1.1066149880163654, 0.3...
29993312    [[1.921257065848562, -0.5309580322614332, 2.45...
29996513    [[0.6878757132109704, -0.34900547445278896, 0....
29998399    [[-0.751069198199553, -0.16705291664414468, -0...
Length: 7853, dtype: object

### Dynamic val preprocessing

In [9]:
id_lengths_val = dynamic_val['id'].value_counts().to_dict()
dynamic_val = dynamic_val.sort_values(by=['id', 'charttime'])
dynamic_val = dynamic_val.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_val['id']).agg(list)

dynamic_val

id
20015730    [[-0.9566327569724848, 0.37880475678178827, -0...
20032048    [[-0.33994208065368914, 0.19685219897314393, -...
20048588    [[0.6878757132109704, -0.16705291664414468, -0...
20048978    [[0.8934392719839024, -0.7129105900700776, 1.1...
20055299    [[0.2767485956651066, 0.19685219897314393, 0.5...
                                  ...                        
29925024    [[-0.545505639426621, -0.34900547445278896, -0...
29926463    [[0.8934392719839024, -1.6226733791132992, -0....
29953754    [[0.8934392719839024, -0.34900547445278896, -0...
29989089    [[-0.751069198199553, 0.7427098723990768, -1.1...
29990494    [[0.6878757132109704, -0.5309580322614332, -0....
Length: 873, dtype: object

### Dynamic test preprocessing

In [10]:
id_lengths_test = dynamic_test['id'].value_counts().to_dict()
dynamic_test = dynamic_test.sort_values(by=['id', 'charttime'])
dynamic_test = dynamic_test.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_test['id']).agg(list)

dynamic_test

id
20003491    [[-0.13437852188075722, -0.34900547445278896, ...
20022095    [[4.1824562123508136, -3.0782938415824534, -0....
20026358    [[1.3045663895297661, -1.4407208213046547, 0.4...
20030062    [[-0.33994208065368914, 0.014899641164499634, ...
20042619    [[-0.33994208065368914, 0.5607573145904325, -0...
                                  ...                        
29954601    [[0.8934392719839024, 0.37880475678178827, 0.3...
29966553    [[-0.545505639426621, 0.19685219897314393, -0....
29967192    [[0.0711850368921747, -0.7129105900700776, -0....
29994296    [[-0.545505639426621, -0.7129105900700776, 1.4...
29997500    [[-0.751069198199553, 2.7441880082941643, -0.5...
Length: 970, dtype: object

In [11]:
notes = notes[['id', 'charttime', 'text', 'interval']]

notes_train = notes[notes['id'].isin(static_train['id'])].copy()
notes_val = notes[notes['id'].isin(static_val['id'])].copy()
notes_test = notes[notes['id'].isin(static_test['id'])].copy()

In [12]:
train_data = MultimodalDataset(static=static_train, dynamic=dynamic_train, id_lengths=id_lengths_train, notes=notes_train)
validation_data = MultimodalDataset(static=static_val, dynamic=dynamic_val, id_lengths=id_lengths_val, notes=notes_val)

train_loader = DataLoader(train_data, batch_size=2000, shuffle=True, collate_fn=collation)
val_loader = DataLoader(validation_data, batch_size=400, shuffle=True, collate_fn=collation)

In [13]:
text_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

for params in text_model.parameters():
    params.requires_grad = False

model = LOSNetWeighted(input_size=9, out_features=1, hidden_size=128, text_model=text_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f'device: {device}')

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device: cuda


In [14]:
for name, param in model.text_model.named_parameters():
    print(f"Layer: {name}, Frozen: {not param.requires_grad}\n")

Layer: embeddings.word_embeddings.weight, Frozen: True

Layer: embeddings.position_embeddings.weight, Frozen: True

Layer: embeddings.token_type_embeddings.weight, Frozen: True

Layer: embeddings.LayerNorm.weight, Frozen: True

Layer: embeddings.LayerNorm.bias, Frozen: True

Layer: encoder.layer.0.attention.self.query.weight, Frozen: True

Layer: encoder.layer.0.attention.self.query.bias, Frozen: True

Layer: encoder.layer.0.attention.self.key.weight, Frozen: True

Layer: encoder.layer.0.attention.self.key.bias, Frozen: True

Layer: encoder.layer.0.attention.self.value.weight, Frozen: True

Layer: encoder.layer.0.attention.self.value.bias, Frozen: True

Layer: encoder.layer.0.attention.output.dense.weight, Frozen: True

Layer: encoder.layer.0.attention.output.dense.bias, Frozen: True

Layer: encoder.layer.0.attention.output.LayerNorm.weight, Frozen: True

Layer: encoder.layer.0.attention.output.LayerNorm.bias, Frozen: True

Layer: encoder.layer.0.intermediate.dense.weight, Frozen: True

In [15]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
    
    def forward(self, predicted, actual):
        return torch.sqrt(self.mse(predicted, actual))

In [16]:
criterion = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [17]:
!nvidia-smi

Sun Apr  7 09:19:25 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   49C    P2    73W / 300W |   3279MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
loss_base_path = 'losses/bert-frozen-all/with-outliers/combined_regression'
writer = SummaryWriter('tensorboard/runs/with-outliers/combined_regression')

In [19]:
epochs = 200
training_loss = []
validation_loss = []
patience = 10
stagnation = 0

for epoch in range(1, epochs+1):
    print(f'training epoch: [{epoch}/{epochs}]')
    model.train()
    training_loss_epoch = 0

    for step, batch in enumerate(train_loader):
        packed_dynamic_X, notes_X, notes_intervals, los = batch

        packed_dynamic_X = packed_dynamic_X.to(device)
        los = los.to(device)

        notes_X_gpu = []
        for notes in notes_X:
            notes_gpu = {key: value.to(device) for key, value in notes.items()}
            notes_X_gpu.append(notes_gpu)

        outputs = model(packed_dynamic_X, notes_X_gpu, notes_intervals)

        loss = criterion(outputs, los)
        writer.add_scalar('Loss/train', loss.item(), epoch * len(train_loader) + step)
        training_loss_epoch += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % max(1, round(len(train_loader) * 0.1)) == 0:
            print(f'step: [{step+1}/{len(train_loader)}] | loss: {loss.item():.4}')

            if step+1 == 1 and epoch == 1:
                with open(f'{loss_base_path}/loss_step.txt', 'w') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

            else:
                with open(f'{loss_base_path}/loss_step.txt', 'a') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

    avg_training_loss_epoch = training_loss_epoch / len(train_loader)
    writer.add_scalar('Loss/train_avg', avg_training_loss_epoch.item(), epoch)

    training_loss.append(avg_training_loss_epoch.item())
    print(f'Training epoch loss: {avg_training_loss_epoch.item():.4f}\n')

    if epoch == 1:
        with open(f'{loss_base_path}/training_loss_epoch.txt', 'w') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    else:
        with open(f'{loss_base_path}/training_loss_epoch.txt', 'a') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    print(f'validation epoch: [{epoch}/{epochs}]')
    
    model.eval()
    with torch.no_grad():
        validation_loss_epoch = 0

        for val_step, val_batch in enumerate(val_loader):
            packed_dynamic_X_val, notes_X_val, notes_intervals_val, los_val = val_batch

            packed_dynamic_X_val = packed_dynamic_X_val.to(device)
            los_val = los_val.to(device)

            notes_X_val_gpu = []
            for notes in notes_X_val:
                notes_val_gpu = {key: value.to(device) for key, value in notes.items()}
                notes_X_val_gpu.append(notes_val_gpu)

            val_outputs = model(packed_dynamic_X_val, notes_X_val_gpu, notes_intervals_val)
            val_loss = criterion(val_outputs, los_val)

            writer.add_scalar('Loss/val', val_loss.item(), epoch * len(val_loader) + val_step)
            validation_loss_epoch += val_loss

        avg_validation_loss = validation_loss_epoch / len(val_loader)
        writer.add_scalar('Loss/val_avg', avg_validation_loss.item(), epoch)
        print(f'Validation epoch loss: {avg_validation_loss.item():.4f}\n')
        
        if len(validation_loss) == 0 or (avg_validation_loss.item() < min(validation_loss)):
            stagnation = 0
            torch.save(model.state_dict(), 'saved-models/no-outliers/bert_frozen_all_combined_regression_best_model.pth')
            print(f'new minimum validation loss')
            print(f'model saved\n')

        else:
            stagnation += 1

        validation_loss.append(avg_validation_loss.item())

        if epoch == 1:
            with open(f'{loss_base_path}/validation_loss_epoch.txt', 'w') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_validation_loss.item():.4f}\n')

        else:
            with open(f'{loss_base_path}/validation_loss_epoch.txt', 'a') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_validation_loss.item():.4f}\n')

        if stagnation >= patience:
            print(f'No improvement over {patience} epochs')
            print('Early stopping\n')
            break

    model.train()

    print('===============================\n')

writer.close()
print(f'min training loss: {min(training_loss):.4f}')
print(f'min validation loss: {min(validation_loss):.4f}')
    

training epoch: [1/200]
step: [1/4] | loss: 8.444
step: [2/4] | loss: 7.776
step: [3/4] | loss: 8.59
step: [4/4] | loss: 8.211
Training epoch loss: 8.2550

validation epoch: [1/200]
Validation epoch loss: 8.7986

new minimum validation loss
model saved


training epoch: [2/200]
step: [1/4] | loss: 7.988
step: [2/4] | loss: 8.184
step: [3/4] | loss: 8.144
step: [4/4] | loss: 8.099
Training epoch loss: 8.1036

validation epoch: [2/200]
Validation epoch loss: 8.5388

new minimum validation loss
model saved


training epoch: [3/200]
step: [1/4] | loss: 8.152
step: [2/4] | loss: 7.706
step: [3/4] | loss: 8.008
step: [4/4] | loss: 7.934
Training epoch loss: 7.9500

validation epoch: [3/200]
Validation epoch loss: 7.5382

new minimum validation loss
model saved


training epoch: [4/200]
step: [1/4] | loss: 7.747
step: [2/4] | loss: 7.873
step: [3/4] | loss: 7.993
step: [4/4] | loss: 7.594
Training epoch loss: 7.8016

validation epoch: [4/200]
Validation epoch loss: 7.9408


training epoch: [5